# Use google drive

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import library

In [0]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

# Parameter

In [0]:
TARGET_SIZE = (299, 299) #fixed size for InceptionV3
NB_EPOCHS = 20
BATCH_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172
n_classes = 196
nb_train_samples = 8144
nb_val_samples = 8044
base_path = '/content/drive/My Drive/computer vision/stanford-car-dataset-by-classes-folder'
#base_path = ''
data_dir = base_path + '/car_data'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

In [25]:
train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True)

test_datagen = ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size=TARGET_SIZE,
  batch_size=BATCH_SIZE)

validation_generator = test_datagen.flow_from_directory(
  test_dir,
  target_size=TARGET_SIZE,
  batch_size=BATCH_SIZE)

Found 8144 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


# Fune-tuning
## base model

In [26]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
predictions = Dense(n_classes, activation='softmax')(x) #new softmax layer
model = Model(input=base_model.input, output=predictions)
for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
    layer.trainable = False
for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
    layer.trainable = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, None, None, 3 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, None, None, 3 0           batch_normalization_189[0][0]    
____________________________________________________________________________________________

# Train model

In [0]:
# load the model
try:
    model.load_weights(base_path+"model.h5")
except:
    print("No model is loaded.")

## Use checkpoint to save progress

In [0]:
filepath=base_path+"model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [36]:
# fit the model
history_tl = model.fit_generator(train_generator,
                                nb_epoch=10,
                                samples_per_epoch=nb_train_samples,
                                class_weight='auto',
                                callbacks = callbacks_list, 
                                verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., class_weight="auto", callbacks=[<keras.ca..., verbose=1, steps_per_epoch=254, epochs=10)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/10
254/254 [==============================] - 225s 887ms/step - loss: 0.3496 - acc: 0.9414

Epoch 00001: acc improved from -inf to 0.94169, saving model to /content/drive/My Drive/computer vision/stanford-car-dataset-by-classes-folder/model.h5
Epoch 2/10
254/254 [==============================] - 230s 906ms/step - loss: 0.3400 - acc: 0.9393

Epoch 00002: acc did not improve from 0.94169
Epoch 3/10
254/254 [==============================] - 234s 921ms/step - loss: 0.3520 - acc: 0.9347

Epoch 00003: acc did not improve from 0.94169
Epoch 4/10
254/254 [==============================] - 229s 903ms/step - loss: 0.3402 - acc: 0.9396

Epoch 00004: acc did not improve from 0.94169
Epoch 5/10
254/254 [==============================] - 226s 889ms/step - loss: 0.3301 - acc: 0.9422

Epoch 00005: acc improved from 0.94169 to 0.94231, saving model to /content/drive/My Drive/computer vision/stanford-car-dataset-by-classes-folder/model.h5
Epoch 6/10
254/254 [==============================] - 2

# Evaluate model

In [37]:
evalute = model.evaluate_generator(validation_generator,
                                   verbose=1)
print(evalute[1])

252/252 [==============================] - 221s 879ms/step
0.7194378808754147
